## 本部分简介
- 文件操作

[本部分参考网页](http://www.cnblogs.com/alex3714/articles/5717620.html)

## 文件
程序与文件进行数据交互

**文件操作流程**：
- 打开文件，得到文件句柄并赋值给一个变量
- 通过句柄对文件进行操作
- 关闭文件 

**文件打开**：使用open()命令,参数为文件名称、打开模式、编码集等

In [15]:
# windows 默认是gbk格式。Python3默认是utf-8。如果不指定编码集，则以默认的gbk编码打开，会报错。
f = open("lyric.txt",'r',encoding = "utf-8")  # 得到文件句柄，包括文件名、硬盘起始位置,读写方式等信息，默认打开方式为只读。
f
# print(f.read())  # 打印文件内容

<_io.TextIOWrapper name='lyric.txt' mode='r' encoding='utf-8'>

**文件打开模式**：

- r，只读模式（默认）。
- w，只写模式。【不可读；不存在则创建；存在则删除内容；】
- a，追加模式。【不可读；不存在则创建；存在则只追加内容；】
- 
- "+" 表示可以同时读写某个文件。【写的时候只会从文件最后进行内容追加，不能加到中间部分】
 -   r+，可读写文件。【可读；可写；可追加】。【最常用的】。起始时指针在文件开头。
 -   w+，写读：该模式会首先根据w命令创建文件，已有文件删除。与读写模式（r+）的情况不同。用得不多。
 -   a+，追加。【可读；可写；可追加】。起始时指针在文件末尾。
- 
- "U"表示在读取时，可以将 \r \n \r\n自动转换成 \n （与 r 或 r+ 模式同使用）。因为在Windows上以\r\n换行。Linux上以\n换行。
 -   rU
 -   r+U
- 
- "b"表示处理二进制文件（如：FTP发送上传ISO镜像文件，linux可忽略，windows处理二进制文件时需标注）
 -   rb（网络传输时使用）
 -   wb
 -   ab

**文件关闭**：使用close()命令,在使用完文件句柄之后，需要关闭。

In [16]:
f.close()

**实现文件自动关闭**：使用with语句管理上下文,Python3中特有的可以使用一个with对多个文件进行同时管理

In [105]:
# 打开文件，自动关闭
with open("lyric.txt",'r',encoding = "utf-8") as f:  # 相当于 "f = open("lyric.txt",'r',encoding = "utf-8")"
    print(f.readline())

Somehow, it seems the love I knew was always the most destructive kind



In [107]:
# 打开多个文件，，自动关闭
with open("lyric.txt",'r',encoding = "utf-8") as f1, \
        open("my_lyric.txt",'r',encoding = "utf-8") as f2:
    print(f1.readline())
    print(f2.readline())

Somehow, it seems the love I knew was always the most destructive kind

My lyric



**文件读取**：文件指针初始时从头开始，随着读写命令光标移动。

In [32]:
# 打开文件
f = open("lyric.txt",'r',encoding = "utf-8")

# data = f.read(50)  # 读取全部内容，返回字符串，指针指到最后。数字表示读多少个字符
data = f.readline(100)  # 读一行，返回字符串，指针指到第二行开头。数字表示读多少个字符，超出该行字符数则只读一行
# data = f.readlines()  # 读多行，返回列表，每个元素为每行的内容（字符串）。

# 循环读文件，实现每次只读一行，对读大文件读写时效率高
"""
for line in f:  # 这里的f是迭代器的作用
    print(line.strip())
"""

print(data)

# 关闭文件
f.close()

Somehow, it seems the love I knew was always the most destructive kind



**写文件**：不同模式下光标起始位置不同。

In [57]:
# 写普通文件

# 打开文件
f = open("my_lyric.txt",'w',encoding = "utf-8")

f.write('My lyric\n')  # 直接写字符串
f.writelines(["菜鸟教程 1\n", "菜鸟教程 2"])  # 写列表中的多个字符串

# 关闭文件
f.close()

In [53]:
# 写文件时的指针变化

# 打开文件
f = open("lyric.txt",'r+',encoding = "utf-8")

print('起始位置-->'.ljust(20,'>'),f.tell())
print(f.readline().strip())
print('读一行后位置-->'.ljust(20,'>'),f.tell())
f.write('\nMy File')
print('写一行后位置-->'.ljust(20,'>'),f.tell())
f.seek(256)  # 返回到某个位置。参数n表示指针返回到第n个字符之后。
print('到256后位置-->'.ljust(20,'>'),f.tell())
print(f.readline().strip())
print('再读一行后位置-->'.ljust(20,'>'),f.tell())

# 关闭文件
f.close()

起始位置-->>>>>>>>>>>>>> 0
Somehow, it seems the love I knew was always the most destructive kind
读一行后位置-->>>>>>>>>>>> 71
写一行后位置-->>>>>>>>>>>> 2604
到256后位置-->>>>>>>>>>> 256
my tongue
再读一行后位置-->>>>>>>>>>> 266


**二进制文件读写**：

In [69]:
# 打开二进制文件，以读写二进制的方式打开
f = open("my_lyric_binary.txt",'rb+')

f.write('My File\n'.encode())  # 默认从utf-8 encode 成二进制写入

f.seek(0)
print(f.readline())

f.seek(0)
print(f.readline().decode())  # 默认 decode 成 utf-8 格式

# 关闭文件
f.close()

b'My File\n'
My File



**文件修改**：是通过“先将文件读出来，写到内存，修改后再写到文件中”的方式修改。

In [83]:
# 修改文件的例子
# 实现将my_lyric.txt文件中所有有文字“教程”的行，最前面加“--> ”符号
import os

file_orignal = open('my_lyric.txt','r',encoding = 'utf-8')
file_new = open('my_lyric_new.txt','w',encoding = 'utf-8')

for data in file_orignal:
    if "教程" in data:
        data = "{} {}".format("-->",data)
    file_new.write(data)

file_orignal.close()
file_new.close()

os.remove('my_lyric.txt')  # 文件删除
os.rename('my_lyric_new.txt','my_lyric.txt')  # 文件重命名，当原文件存在时，不能重命名

**文件指针相关**：seek与tell函数

In [31]:
# 打开文件
f = open("lyric.txt",'r',encoding = "utf-8")

print(f.tell())  # 打印文件当前指针位置，表示前方有多少字符
f.readline()
print(f.tell())
f.seek(456)  # 返回到某个位置。参数n表示指针返回到第n个字符之后。
print(f.tell())

# 关闭文件
f.close()

0
71
456


**其他命令**：

In [34]:
# f.flush() 命令，写文件时系统等到缓存满了才写到文件中（一般在dos命令行中有这种缓存机制），使用flush命令进行强制刷新，不等缓存满了才写文件
# f.buffer 命令，返回文件编码方式
# f.detach() 命令，在编辑时转换编码方式
# f.encoding 命令，返回文件编码方式
# f.errors 命令，异常处理时使用
# f.fileno() 命令，返回句柄f在内存中的编号
# f.name 命令，返回文件名称
# f.isatty() 命令，是否是终端设备
# f.seekable() 命令，判断是否可移动指针
# f.readable() 命令，判断是否可读
# f.writeable() 命令，判断是否可写
# f.closed 命令，判断是否关闭
# f.truncate(n) 命令，截断。如果不写参数n则清空文件。参数n表示保留前n个字符，与指针当前位置无关

**进度条的小例子**：使用flush命令输出进度条效果

In [41]:
import sys,time
for i in range(20):
    sys.stdout.write('#')  # 标准输出方式，与print不同
    sys.stdout.flush()  # 强制输出，没有的话会某几个‘#’同时输出
    time.sleep(0.1)  # 停顿0.1s 

####################

## 作业四：修改haproxy配置文件 
任务说明：
- 